## Problema de Deep Learning com Aprendizado por Transferência

In [4]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os

In [2]:
train_dir = 'train'
test_dir = 'test'

# Pré-processamento e aumento de dados
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
# Geradores de imagens
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 25000 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications.vgg16 import preprocess_input

test_images = [os.path.join(test_dir, img) for img in os.listdir(test_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array

X_test = np.array([load_and_preprocess_image(img) for img in test_images])


In [6]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Saída binária
])

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)


58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=train_generator
)


Epoch 1/10
148/782 [====>.........................] - ETA: 26:52 - loss: 0.5057 - accuracy: 0.7373

In [ ]:
predictions = model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)